In [19]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
import cv2
from tensorflow.keras.models import load_model
import numpy as np
import os

In [20]:
prototxt=os.path.sep.join([r'C:\Users\AAVEJ\Desktop\Mtech\MLP_project\face_detector','deploy.prototxt'])
weights=os.path.sep.join([r'C:\Users\AAVEJ\Desktop\Mtech\MLP_project\face_detector','res10_300x300_ssd_iter_140000.caffemodel'])

In [21]:
prototxt

'C:\\Users\\AAVEJ\\Desktop\\Mtech\\MLP_project\\face_detector\\deploy.prototxt'

In [22]:
weights

'C:\\Users\\AAVEJ\\Desktop\\Mtech\\MLP_project\\face_detector\\res10_300x300_ssd_iter_140000.caffemodel'

In [23]:
net=cv2.dnn.readNet(prototxt,weights)

In [24]:
model=load_model(r'C:\Users\AAVEJ\Desktop\Mtech\MLP_project\umar.model')

In [716]:
image=cv2.imread(r'C:\Users\AAVEJ\Desktop\Mtech\MLP_project\exp\69.jpeg')

In [717]:
image

array([[[207, 175, 134],
        [207, 175, 134],
        [207, 175, 134],
        ...,
        [250, 251, 249],
        [250, 251, 249],
        [250, 251, 249]],

       [[207, 175, 134],
        [207, 175, 134],
        [207, 175, 134],
        ...,
        [250, 251, 249],
        [250, 251, 249],
        [250, 251, 249]],

       [[207, 175, 134],
        [207, 175, 134],
        [207, 175, 134],
        ...,
        [250, 251, 249],
        [250, 251, 249],
        [250, 251, 249]],

       ...,

       [[114, 135, 162],
        [114, 135, 162],
        [114, 135, 162],
        ...,
        [ 90, 115, 149],
        [ 88, 115, 149],
        [ 88, 115, 149]],

       [[114, 135, 162],
        [114, 135, 162],
        [113, 134, 161],
        ...,
        [ 88, 115, 149],
        [ 88, 115, 149],
        [ 88, 115, 149]],

       [[114, 135, 162],
        [114, 135, 162],
        [113, 134, 161],
        ...,
        [ 88, 115, 149],
        [ 88, 115, 149],
        [ 88, 115, 149]]

In [718]:
(h,w)=image.shape[:2]

In [719]:
(h,w)

(1280, 960)

In [720]:
blb=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [721]:
blb

array([[[[103., 103., 103., ..., 145., 146., 146.],
         [103., 103., 103., ..., 144., 146., 146.],
         [103., 103., 103., ..., 143., 145., 146.],
         ...,
         [ 12.,  13.,  10., ..., -11., -10.,  -9.],
         [ 12.,  10.,   9., ..., -12., -13., -14.],
         [ 10.,   9.,   8., ..., -14., -15., -16.]],

        [[ -2.,  -2.,  -2., ...,  73.,  74.,  74.],
         [ -2.,  -2.,  -2., ...,  72.,  74.,  74.],
         [ -2.,  -2.,  -2., ...,  71.,  73.,  74.],
         ...,
         [-40., -39., -42., ..., -64., -63., -60.],
         [-40., -42., -43., ..., -64., -64., -62.],
         [-42., -43., -44., ..., -65., -63., -62.]],

        [[ 11.,  11.,  11., ..., 125., 126., 126.],
         [ 11.,  11.,  11., ..., 124., 126., 126.],
         [ 11.,  11.,  11., ..., 123., 125., 126.],
         ...,
         [ 41.,  42.,  39., ...,  23.,  26.,  29.],
         [ 41.,  39.,  38., ...,  22.,  25.,  26.],
         [ 39.,  38.,  37., ...,  21.,  25.,  26.]]]], dtype=float32)

In [722]:
blb.shape

(1, 3, 300, 300)

In [723]:
net.setInput(blb)
detections=net.forward()

In [724]:
detections

array([[[[0.        , 1.        , 0.9982488 , ..., 0.21405923,
          0.69405013, 0.5996343 ],
         [0.        , 1.        , 0.12881123, ..., 0.7457616 ,
          0.692359  , 0.798765  ],
         [0.        , 1.        , 0.12616992, ..., 0.76302   ,
          0.35799852, 0.8025866 ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [725]:
detections.shape[2]

200

In [726]:
for i in range (0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    
    if confidence > 0.5:
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        (startX,startY)=(max(0,startX), max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        face=image[startY:endY , startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        
        (mask,withoutmask)=model.predict(face)[0]
        
        
        label='Mask' if mask>withoutmask else 'No mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        label="{}: {:.2f}%".format(label,max(mask,withoutmask)*100)
        
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
        

cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyWindow()
        
        
        
        

TypeError: destroyWindow() missing required argument 'winname' (pos 1)